In [2]:
# -*- coding: utf-8 -*-

import os
import sys
import gc
import joblib
sys.path.append(os.path.join('..', '2. FE'))
import FE_innData as FEin
import FE_extData as FEex

import numpy as np
import pandas as pd
import argparse
from tqdm import tqdm
from datetime import datetime

In [57]:
import imp
imp.reload(FEin)

<module 'FE_innData' from '../2. FE/FE_innData.py'>

In [58]:
df = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', '2020 빅콘테스트 데이터분석분야-챔피언리그_방송편성표추천데이터.xlsx'))

In [59]:
df

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2020-01-01 06:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-01 07:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01 08:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-01 09:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-01 10:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-01-01 11:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-01-01 12:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-01-01 13:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-01-01 14:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-01-01 15:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
df['방송날'] = df['방송일시'].dt.date    
df['방송년도'] = df['방송일시'].dt.year
df['방송월'] = df['방송일시'].dt.month
df['방송일'] = df['방송일시'].dt.day
df['방송시간(시간)'] = df['방송일시'].dt.hour
df['방송시간(분)'] = df['방송일시'].dt.minute

In [36]:
# 날씨 데이터 FE
w_19 = pd.read_csv(os.path.join('..', '..', '0.Data', '03_외부데이터', '2019_weather.csv'), encoding = 'cp949', dtype='unicode')
w_20 = pd.read_csv(os.path.join('..', '..', '0.Data', '03_외부데이터', '2020_weather.csv'), encoding = 'cp949', dtype='unicode')
df_wth = pd.concat([w_19, w_20], axis = 0)
df_wth = FEex.preprocessing_weather(df_wth)

# 미세먼지 데이터
dust_2019 = pd.read_csv(os.path.join('..', '..', '0.Data', '03_외부데이터', '2019_dust.csv'), encoding = 'cp949')
dust_2020 = pd.read_csv(os.path.join('..', '..', '0.Data', '03_외부데이터', '2020_dust.csv'), encoding = 'cp949')
df_dust = pd.concat([dust_2019, dust_2020], axis = 0)
df_dust = FEex.preprocessing_dust(df_dust, 'train')

In [61]:
df = df.merge(df_wth, left_on = ['방송년도', '방송월', '방송일', '방송시간(시간)'], right_on = ['연도', '월', '일' ,'시간'], how = 'left').drop(['연도', '월', '일', '시간'], axis = 1)
df = df.merge(df_dust, left_on = ['방송년도', '방송월', '방송일', '방송시간(시간)'], right_on = ['연도', '월', '일', '시간'], how ='left').drop(['연도', '월', '일', '시간'], axis = 1)

In [62]:
df_eco = FEex.preprocessing_economy()
# 외부 데이터 merge
df = df.merge(df_eco, left_on = ['방송년도', '방송월'], right_on = ['연도', '월'], how = 'left').drop(['연도', '월'], axis = 1)

In [63]:
df = FEin.engineering_DatePrice(df, 'recommend')

In [90]:
df = df[list(df.columns[:1]) + list(df.columns[8:14]) + list(df.columns[1:8]) + list(df.columns[14:])]

In [96]:
[df.columns[0]]

['방송일시']